## Install required packages

In [ ]:
!pip install diffusers transformers accelerate opencv-python moviepy --upgrade


## Create SDXL-Turbo pipeline

In [ ]:
import os
import cv2
import torch
import shutil

from diffusers import AutoPipelineForText2Image

from IPython.display import Video, display, clear_output
from moviepy.editor import VideoFileClip, CompositeVideoClip

pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16")
pipe = pipe.to("cuda")


## Helper functions to generate frames and create video

In [ ]:
def generate_frames(prompts, frames_path, seed):
    frames = []
    cleanup_path("_frames")
    for n, prompt in enumerate(prompts):
        generator = torch.Generator().manual_seed(seed)
        image = pipe(
            prompt=prompt,
            generator=generator,
            num_inference_steps=1,
            guidance_scale=0.0,
        ).images[0]
        frame_path = f"{frames_path}/frame{n:04}.jpg"
        frames.append(frame_path)
        image.save(frame_path)
        clear_output(wait=True)
        display(image)
    return frames

def create_video(folder_path, output_video_file, fps=25, size=(512, 512)):
    images = [img for img in os.listdir(folder_path) if img.endswith(".jpg") or img.endswith(".png")]
    images.sort()
    video = cv2.VideoWriter(output_video_file, cv2.VideoWriter_fourcc(*'mp4v'), fps, size)
    for image in images + images[::-1]:
        img_path = os.path.join(folder_path, image)
        img = cv2.imread(img_path)
        img = cv2.resize(img, size)
        video.write(img)
    video.release()

def cleanup_path(frames_path):
    try: shutil.rmtree(frames_path)
    except: pass
    os.mkdir(frames_path)


## Compose prompt list to generate frames

In [ ]:
%%time

video_path1 = "video1.mp4"
prompts = [f"ultra realistic, hi resolution portrait of a male with {n} years old" for n in range(10, 60)]
generate_frames(prompts, "_frames", 44)
create_video("_frames", video_path1)

video_path2 = "video2.mp4"
prompts = [f"ultra realistic, hi resolution portrait of a female with {n} years old" for n in range(10, 60)]
generate_frames(prompts, "_frames", 44)
create_video("_frames", video_path2)


## Create video from frames

In [ ]:
video = VideoFileClip(video_path1)
display(video.ipython_display())
video = VideoFileClip(video_path2)
display(video.ipython_display())
